In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import BertModel, BertPreTrainedModel, Trainer, TrainingArguments
from torch.utils.data import DataLoader

In [2]:
from transformers import BertForSequenceClassification, AlbertForSequenceClassification
from transformers import LongformerForSequenceClassification, LongformerTokenizer, Trainer, TrainingArguments

l = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', num_labels=2)
m = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
n = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=2)


config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--allenai--longformer-base-4096. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
m

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [4]:
l

LongformerForSequenceClassification(
  (longformer): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0-11): 12 x LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
          

In [5]:
total_params = sum(m.numel() for m in m.parameters())
print(total_params)
total_params = sum(l.numel() for l in n.parameters())
print(total_params)

109483778
11685122


In [ ]:
import sys
sys.path.append('C:\\Users\\user\\Programming\\Git\\challenge_data\\')
from PreprocessData import *

train_file_path = 'C:\\Users\\user\\Programming\\Git\\challenge_data\\train_tweets\\ArgentinaBelgium72.csv'
train_data = process_csv(train_file_path, 128, False)

class CustomDataset(Dataset):
    def __init__(self, tweets, labels):
        self.tweets = [torch.tensor(tweet, dtype=torch.long) for tweet in tweets]
        self.labels = torch.tensor(labels, dtype=torch.long)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return {
            "input_ids": self.tweets[idx],  # Required key for BERT input
            "attention_mask": torch.ones_like(self.tweets[idx]),  # Add attention mask
            "labels": self.labels[idx],  # Required key for classification
        }

dataset_train = CustomDataset(train_data['Tweet'], train_data['EventType'])

In [ ]:
import torch
import numpy as np
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset


model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2  # Set num_labels to 2 for binary classification
)


for param in model.bert.parameters():
    param.requires_grad = False


training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy='no',
    gradient_accumulation_steps=2  # Accumulate gradients for larger effective batch size
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/7356 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [46]:
class BertForBinaryClassification(BertPreTrainedModel):
    def __init__(self, config, classifier_hidden_sizes=[256]):
        super(BertForBinaryClassification, self).__init__(config)

        self.bert = BertModel(config)

        classifier_layers = []
        input_size = config.hidden_size

        for hidden_size in classifier_hidden_sizes:
            classifier_layers.append(nn.Linear(input_size, hidden_size))
            classifier_layers.append(nn.ReLU())
            classifier_layers.append(nn.Dropout(config.hidden_dropout_prob))
            input_size = hidden_size

        classifier_layers.append(nn.Linear(input_size, 1))
        self.classifier = nn.Sequential(*classifier_layers)

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
        )

        pooled_output = outputs.pooler_output
        logits = self.classifier(pooled_output)
        probs = torch.sigmoid(logits)

        loss = None
        if labels is not None:
            loss_fct = nn.BCEWithLogitsLoss()
            # Преобразуем метки в тип float и изменяем размерность
            labels = labels.float().view(-1, 1)
            loss = loss_fct(logits, labels)

        output = (probs,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output



from transformers import BertConfig
config = BertConfig.from_pretrained('bert-base-uncased')



classifier_hidden_sizes = [512, 256, 128]

model = BertForBinaryClassification(
    config,
    classifier_hidden_sizes=classifier_hidden_sizes
)



for param in model.bert.parameters():
    param.requires_grad = False



training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy='no',
    gradient_accumulation_steps=2
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train
)

trainer.train()


c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/7356 [00:00<?, ?it/s]

KeyboardInterrupt: 